Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos <br>
Primer Semestre 2025
<br>

<h1><center>Tarea 2: Isolation Forest, HAC, DBSCAN y HDBSCAN </center></h1>
        Profesor: Marcelo Mendoza<br>
        Ayudantes encargados: Itzae Flores, Claudia González y Eduardo Salinas.<br>
        Fecha de entrega: 23 de abril del 2025.

## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas.

**IMPORTANTE**:
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas.
- El notebook debe tener todas las celdas de código ejecutadas.
- El carácter de esta tarea es **INDIVIDUAL**. Cualquier instancia de copia resultará en un 1,1 como nota de curso.

---

## Librerías

In [1]:
##Importa acá las librerias que vayas a utilizar
## De ser necesario, utiliza el comando !pip install a aquellas librerias o modulos que no se encuentren instalados, como puede ser HDBSCAN.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from random import random
from matplotlib import pyplot as plt

from scipy.cluster.hierarchy import dendrogram
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, minmax_scale
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN, cluster_optics_dbscan
from sklearn.ensemble import IsolationForest
# from sklearn.cluster import , HDBSCAN
from sklearn.mixture import GaussianMixture


El siguiente codigo sirve para importar HDBSCAN en el caso de que les arroje error importar desde sklearn.

In [2]:
from hdbscan import HDBSCAN

# Introducción

El conjunto de datos corresponde a características extraidas de imagenes de distintos clientes que están buscando pedir un crédito a un banco. Se trabaja con este dataset para predecir si el cliente aceptó o no el último préstamo ofrecido (dado por la columna "Personal Loan") por medio del uso de HAC, DBSCAN, HDBSCAN e Isolation Forest.

En esta tarea deberás trabajar con el dataset Imbalanced Personal Bank Loan Classification que puedes descargar de [Kaggle] (https://www.kaggle.com/code/farzadnekouei/imbalanced-personal-bank-loan-classification) o Canvas.

# Parte 1: Carga y Preprocesamiento de Datos (9 puntos)

En esta primera parte se debe cargar el dataset y luego describir el comportamiento de algunas variables de interés. Posteriormente, debe realizar el proceso de normalización de los datos.

### 1.1 Carga de datos (1 puntos)

In [11]:
df = pd.read_csv('data.csv')
df.head(5)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


### 1.2 Descripcion del Dataset (5 puntos)

#### 1.2.1 Describa brevemente cuantas columnas tiene el dataset y en que consisten las columna del dataset entregado (1 puntos)

In [17]:
# Descripción de las columnas:

for column in df.columns:
    print(f"{column}: {df[column].dtype}")
    print(df[column].describe())
    print("\n")


ID: int64
count    5000.000000
mean     2500.500000
std      1443.520003
min         1.000000
25%      1250.750000
50%      2500.500000
75%      3750.250000
max      5000.000000
Name: ID, dtype: float64


Age: int64
count    5000.000000
mean       45.338400
std        11.463166
min        23.000000
25%        35.000000
50%        45.000000
75%        55.000000
max        67.000000
Name: Age, dtype: float64


Experience: int64
count    5000.000000
mean       20.104600
std        11.467954
min        -3.000000
25%        10.000000
50%        20.000000
75%        30.000000
max        43.000000
Name: Experience, dtype: float64


Income: int64
count    5000.000000
mean       73.774200
std        46.033729
min         8.000000
25%        39.000000
50%        64.000000
75%        98.000000
max       224.000000
Name: Income, dtype: float64


ZIP Code: int64
count     5000.000000
mean     93152.503000
std       2121.852197
min       9307.000000
25%      91911.000000
50%      93437.000000
75%   

**Respuesta:**

#### 1.2.2 ¿Es posible en este dataset eliminar columnas sin afectar de manera significativa los datos? ¿Por qué? (2 puntos)

**Respuesta:**

En primer lugar podemos asumir que la columna ID la podemos eliminar ya no debería afectar al análisis de dataset debido a que es un dato de identificación que no aporta información adicional.

Luego, dependiendo de los objetivos del análisis podemos determinar que columnas podemos eliminar según la varianza y cantidad de datos que existen, sin embargo por ahora no eliminaremos ninguna otra variable.

#### 1.2.3 ¿Por qué crees que es relevante trabajar estos datos? Averigua 3 posibles usos en que trabajar los datos de esta forma podría ser útil para problemas que resuelven las organizaciones (2 puntos)

**Respuesta:**

Estos datos son importantes de trabajar ya que permiten a las instituciones bancarias tomar decisiones con mayor información respecto a la segmentación de clientes para estrategias personalizadas, identificación de anomalías o comportamientos atípicos y predicción de aceptación de préstamos.

Entrando más en detalle con cada punto lo podemos ver como:

1. Segmentación de clientes para estrategias personalizadas:
A través de técnicas de agrupamiento como HAC o DBSCAN, el banco puede clasificar a sus clientes en distintos segmentos según variables como edad, ingreso, nivel de educación, uso de productos financieros, entre otros. Esto permite ofrecer productos adaptados a cada grupo, optimizando las campañas comerciales y mejorando la tasa de aceptación de los préstamos.

2.	Identificación de anomalías o comportamientos atípicos:
Utilizando algoritmos como HDBSCAN o Isolation Forest, se pueden detectar clientes que presentan patrones significativamente diferentes al resto, lo cual puede ser útil para alertar sobre posibles fraudes, errores en los datos o identificar perfiles poco comunes que requieran análisis más detallado.

3.	Predicción de aceptación de préstamos:
Analizando los patrones históricos, es posible entrenar modelos que anticipen la probabilidad de que un nuevo cliente acepte un préstamo o no pueda pagarlo, esto ayuda a focalizar campañas y acelerar procesos de estimación de riesgo.

**Respuesta:**

### 1.3 Determinación de valores nulos (1 punto)

Determina si existen datos y/o valores nulos presentes en el dataset. En caso de que existan, realiza el tratamiento pertinente.

In [19]:
# Determinar datos y/o valores nulos

print(df.isnull().sum())

# Eliminar columnas innecesarias


ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64


**Respuesta:**

### 1.4 Manejo del Dataset (2 puntos)

Normaliza los valores de las columnas del dataset. ¿Por qué esto es importante para el análisis?

**Respuesta**

# Parte 2: Isolation Forest (15 puntos)

### 2.1 Reduzca la cantidad de componente reteniendo el 95% de la varianza (1 punto)

### 2.2 Utilizando 100 iTrees, cree una instancia de Isolation Forest. ¿Cuántos outliers detecta Isolation Forest y a qué clases verdaderas pertenecen? (3 puntos)

### 2.3 ¿Como cambia el numero de de outliers detectados al variar el parametro ```contamination```? Utilize los valores ```[0.01, 0.05, 0.1, 0.2]``` (3 puntos)


### 2.4 Grafique los "anomaly scores" e indique si se ve un umbral claro (2 puntos)

### 2.5 Que tan estable es la detección de outliers si cambiamos la cantidad de arboles (```n_estimators```) (4 puntos)

### 2.6 Grafique los outliers detectados en las primeras dos componentes principales. (Utilize parametro ```alpha=0.6```) (2 puntos)

# Parte 3: HAC (5 puntos)

### 3.1 Reliza un dendograma sin clusters y con distancia threshold igual a 0 para visualizar los datos (1 punto)

### 3.2 Realiza los Clustering de Agrupación Jerarquica (HAC) para cada distancia de clustering aglomerativo (linkage), prueba utilizando número entre 2 y 10. Calcula el Silhouette para ver cual funcionó mejor y explica el por qué de los resultados. (1.5 puntos)


### 3.3 Ahora, realiza los Clustering de Agrupación Jerarquica (HAC) para cada tipo de linkage. Calcula el Silhouette para ver cual funcionó mejor y explica el por qué de los resultados, mencionando de que manera funciona cada linkage. Usa 20 clusters (1.5 puntos)

### 3.4 Grafica los clusters utilizando un PCA de 2 dimensiones de los datos. Usa 20 clusters y el linkage con mejor silhouette score (1 punto)






Con el análisis realizado, ¿Cuál es la importancia del criterio de vinculación al momento de agrupar los datos? Pruebe ahora utilizando solamente dos clusters y compare los graficos. Debe utilizar al menos un tipo de linkage para hacer la comparación.

# Parte 4: DBSCAN y HDBSCAN (15 puntos)

### 4.1.- DBSCAN

Utilizando DBSCAN, itera sobre valores de eps entre 0.25 y 2, con un salto de 0.25 por iteracion. Ademas, itera sobre el numero minimo de muestras, probando con 1, 2, 3 y 4. (3 puntos)

**RESPUESTA:**

#### 4.1.2 Con la información de los parametros anterior, calcula cual es el número de clusters óptimo para el modelo DBSCAN. (1.5 puntos)

**RESPUESTA:**

#### 4.1.3 Determine tambien el valor optimo del parametro `eps` y `min_samples` (0.5 puntos)

**RESPUESTA:**

#### 4.1.4 Obtenga el numero de clusters utilizando los parametros optimos encontrados. Explique por qué podría haber encontrado ese valor, el cual se puede o no ajustar al número de clases en el dataset. (1 puntos)

**RESPUESTA:**

Se espera un analisis de ajuste de datos a clusters contra las clases. Puntaje por analisis y justificacion.

### 4.2.- HDBSCAN

Utilizando HDSCAN, cree un modelo con el `cluster_selection_epsilon` igual al caso de DBSCAN y determine el número de clusters. Itere sobre el min_cluster_size para encontrar el mejor silhoutte score. Compare el resultado con DBSCAN.  (3 puntos)

**RESPUESTA:**

#### 4.2.2 Grafique los clusters utilizando PCA de 2 dimensiones. (1 punto)

**RESPUESTA:**

¿Que representa cada cluster?

**RESPUESTA:**

### 4.4 Preguntas teoricas sobre HAC, DBSCAN y HDBSCAN (5 puntos)

#### 4.4.1 En tus palabras, ¿Cómo funciona el Agglomerative Clustering? Además, menciona al menos 2 usos para HAC.

**RESPUESTA:**

#### 4.4.2 Explique que representa un dendograma, mencionando cómo las ramas principales podrían significar las clases del datset.

**RESPUESTA:**

#### 4.4.3 En el contexto del modelo DBSCAN, ¿Que es un core sample? ¿Que podría representar un core sample en términos de un dato en el dataset? Apóyese en la clases y sus características.

**RESPUESTA:**

#### 4.4.4 En el modelo de DBSCAN se iteran sobre distintos parametros; eps y min_samples. ¿Qué representan estos parametros?

**RESPUESTA:**

#### 4.4.5 ¿Cuál es la diferencia entre DBSCAN y HDBSCAN? ¿Qué beneficios existen de usar un modelo sobre el otro?

**RESPUESTA:**

**RESPUESTA:**

**Respuesta**

**Para esta sección, utilize el modelo con mejor silhouette score obtenido.**

# Parte 5: Comparación y eficiencia (3 puntos)

#### 5.1 ¿Cuál de los métodos de clustering resultó ser el más eficiente entre HAC, DBSCAN, HDBSCAN o Isolation Forest? Da una explicación de por qué se puede deber el resultado.

**Respuesta**